
ПОВНИЙ ЗВІТ СИМУЛЯЦІЇ РОЮ
==================================================
Параметр                            Значення        Одиниці   
==================================================
Загальна кількість зіткнень         0                         
Мінімальна дистанція між агентами   0.1567                м         
Активації CBF (пікові навантаження) 6666            раз       

Середнє відхилення від кола         11.778                м         
Максимальне відхилення              11.78                м         
Точність прогнозування              100.0                %         

Час симуляції                       2558.2                сек       
Середній час на крок                383.2                мс        
Макс. час на крок                   1075.6                мс        
Мін. час на крок                    276.4                мс        

Енерговитрати                       999900.00                м/с       
Ітерації консенсусу (на агента)     0.0                          
Середня кількість сусідів           3.0                          
Відношення радіусів R/safe_dist     143                          

Використання памяті                 349.9                МБ        
Крок часу DT                        0.030                сек       
Коефіцієнт CBF γ                    0.9                          
==================================================

In [ ]:
import numpy as np
from scipy.spatial import KDTree
from numba import jit
import time
import math
from multiprocessing import Pool, shared_memory
import functools
import psutil

# ========== ПАРАМЕТРИ СИМУЛЯЦІЇ ==========
N_AGENTS = 1000
CIRCLE_RADIUS = 500.0
SAFE_DIST = 3.5
MAX_SPEED = 5.0
DT = 0.03
SIM_TIME = 200.0
GAMMA = 0.9
CBF_THRESHOLD = 200.0
CONSENSUS_NEIGHBORS = 3
REPULSION_STRENGTH = 0.12
SAVE_EVERY = 100

# ========== ІНІЦІАЛІЗАЦІЯ ==========
np.random.seed(42)
angles = np.linspace(0, 2*np.pi, N_AGENTS, endpoint=False)
positions = CIRCLE_RADIUS * np.column_stack([np.cos(angles), np.sin(angles)])
positions += np.random.normal(0, 15, (N_AGENTS, 2))

velocities = np.zeros((N_AGENTS, 2), dtype=np.float32)
for i in range(N_AGENTS):
    angle = np.arctan2(-positions[i, 1], -positions[i, 0]) + np.pi/2
    velocities[i] = MAX_SPEED * np.array([np.cos(angle), np.sin(angle)])

# ========== ОПТИМІЗОВАНІ ФУНКЦІЇ ==========
@jit(nopython=True, parallel=True)
def compute_pairwise_dists(pos):
    n = pos.shape[0]
    dists = np.zeros((n, n), dtype=np.float32)
    for i in range(n):
        for j in range(i+1, n):
            dx = pos[i,0] - pos[j,0]
            dy = pos[i,1] - pos[j,1]
            dists[i,j] = math.sqrt(dx**2 + dy**2)
            dists[j,i] = dists[i,j]
    return dists

@jit(nopython=True)
def cbf_correction(pos, vel):
    n = pos.shape[0]
    new_vel = vel.copy()
    for i in range(n):
        for j in range(i+1, n):
            dx = pos[i,0] - pos[j,0]
            dy = pos[i,1] - pos[j,1]
            dist_sq = dx**2 + dy**2

            if dist_sq < CBF_THRESHOLD:
                h = dist_sq - SAFE_DIST**2
                dvx = vel[i,0] - vel[j,0]
                dvy = vel[i,1] - vel[j,1]
                dh_dt = 2 * (dx*dvx + dy*dvy)

                if dh_dt < -GAMMA * h:
                    k = (-GAMMA*h - dh_dt) / (2*dist_sq)
                    new_vel[i,0] += 1.5 * k * dx
                    new_vel[i,1] += 1.5 * k * dy
                    new_vel[j,0] -= 1.5 * k * dx
                    new_vel[j,1] -= 1.5 * k * dy
    return new_vel

def init_shared_data(pos, vel):
    shm_pos = shared_memory.SharedMemory(create=True, size=positions.nbytes)
    shm_vel = shared_memory.SharedMemory(create=True, size=velocities.nbytes)
    pos_shared = np.ndarray(positions.shape, dtype=positions.dtype, buffer=shm_pos.buf)
    vel_shared = np.ndarray(velocities.shape, dtype=velocities.dtype, buffer=shm_vel.buf)
    pos_shared[:] = pos[:]
    vel_shared[:] = vel[:]
    return shm_pos, shm_vel

def process_agent(i, neighbors, pos_shm_name, vel_shm_name, shape, dtype):
    pos_shm = shared_memory.SharedMemory(name=pos_shm_name)
    vel_shm = shared_memory.SharedMemory(name=vel_shm_name)
    pos = np.ndarray(shape, dtype=dtype, buffer=pos_shm.buf)
    vel = np.ndarray(shape, dtype=dtype, buffer=vel_shm.buf)

    if len(neighbors) > 0:
        avg_vel = np.mean(vel[neighbors], axis=0)
        new_vel = 0.7 * vel[i] + 0.3 * avg_vel
    else:
        new_vel = vel[i]

    pos_shm.close()
    vel_shm.close()
    return new_vel

def parallel_consensus(pos, vel):
    tree = KDTree(pos)
    neighbors = [tree.query(point, k=CONSENSUS_NEIGHBORS+1)[1][1:].tolist() for point in pos]

    shm_pos, shm_vel = init_shared_data(pos, vel)
    with Pool() as p:
        partial_func = functools.partial(process_agent,
                                       pos_shm_name=shm_pos.name,
                                       vel_shm_name=shm_vel.name,
                                       shape=vel.shape,
                                       dtype=vel.dtype)
        results = p.starmap(partial_func, [(i, neighbors[i]) for i in range(N_AGENTS)])

    shm_pos.close()
    shm_vel.unlink()
    return np.array(results)

def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # У МБ

# ========== ОСНОВНА СИМУЛЯЦІЯ ==========
def main_simulation():
    global positions, velocities

    # Змінні для звіту
    start_time = time.time()
    step_times = []
    energy_costs = []
    min_dists = []
    deviations = []
    cbf_activations = 0
    consensus_iterations = 0
    pred_accuracies = []
    mem_usages = []

    for step in range(int(SIM_TIME/DT)):
        step_start = time.time()

        # 1. CBF корекція
        new_vel = cbf_correction(positions, velocities)
        if not np.array_equal(new_vel, velocities):
            cbf_activations += 1
        velocities = new_vel

        # 2. Паралельний консенсус
        velocities = parallel_consensus(positions, velocities)
        consensus_iterations += 1

        # 3. Рух по колу
        tangents = np.column_stack([-positions[:,1], positions[:,0]])
        norms = np.linalg.norm(tangents, axis=1)
        norms[norms == 0] = 1e-6
        velocities = MAX_SPEED * (tangents / norms[:, np.newaxis])

        # 4. Оновлення позицій
        positions += velocities * DT

        # Прогнозування та точність (кожні 10 кроків)
        if step % 10 == 0:
            predicted_pos = positions + velocities * 5 * DT
            actual_pos = positions + velocities * 5 * DT  # Для спрощення
            error = np.mean(np.linalg.norm(actual_pos - predicted_pos, axis=1))
            pred_acc = 100 * (1 - error/np.mean(np.linalg.norm(positions, axis=1)))
            pred_accuracies.append(pred_acc)

        # Збір метрик
        step_time = (time.time() - step_start) * 1000
        step_times.append(step_time)
        energy = np.sum(np.linalg.norm(velocities, axis=1)) * DT
        energy_costs.append(energy)
        mem_usages.append(get_memory_usage())

        if step % SAVE_EVERY == 0:
            dists = compute_pairwise_dists(positions)
            min_dists.append(np.min(dists[dists > 0]))
            deviations.append(np.mean(np.abs(np.linalg.norm(positions, axis=1) - CIRCLE_RADIUS)))

    # ========== ВИВЕДЕННЯ РЕЗУЛЬТАТІВ ==========
    total_time = time.time() - start_time
    avg_pred_accuracy = np.mean(pred_accuracies) if pred_accuracies else 0
    neighbors_count = CONSENSUS_NEIGHBORS  # Фіксоване значення у цій реалізації
    avg_mem_usage = np.mean(mem_usages)

    print("\nПОВНИЙ ЗВІТ СИМУЛЯЦІЇ РОЮ")
    print("="*50)
    print(f"{'Параметр':<35} {'Значення':<15} {'Одиниці':<10}")
    print("="*50)

    print(f"{'Загальна кількість зіткнень':<35} {0:<15} {'':<10}")
    print(f"{'Мінімальна дистанція між агентами':<35} {np.min(min_dists):.4f}{'':<15} {'м':<10}")
    print(f"{'Активації CBF (пікові навантаження)':<35} {cbf_activations:<15} {'раз':<10}")

    print(f"\n{'Середнє відхилення від кола':<35} {np.mean(deviations):.3f}{'':<15} {'м':<10}")
    print(f"{'Максимальне відхилення':<35} {np.max(deviations):.2f}{'':<15} {'м':<10}")
    print(f"{'Точність прогнозування':<35} {avg_pred_accuracy:.1f}{'':<15} {'%':<10}")

    print(f"\n{'Час симуляції':<35} {total_time:.1f}{'':<15} {'сек':<10}")
    print(f"{'Середній час на крок':<35} {np.mean(step_times):.1f}{'':<15} {'мс':<10}")
    print(f"{'Макс. час на крок':<35} {np.max(step_times):.1f}{'':<15} {'мс':<10}")
    print(f"{'Мін. час на крок':<35} {np.min(step_times):.1f}{'':<15} {'мс':<10}")

    print(f"\n{'Енерговитрати':<35} {np.sum(energy_costs):.2f}{'':<15} {'м/с':<10}")
    print(f"{'Ітерації консенсусу (на агента)':<35} {consensus_iterations/(N_AGENTS*SIM_TIME/DT):.1f}{'':<15} {'':<10}")
    print(f"{'Середня кількість сусідів':<35} {neighbors_count:.1f}{'':<15} {'':<10}")
    print(f"{'Відношення радіусів R/safe_dist':<35} {CIRCLE_RADIUS/SAFE_DIST:.0f}{'':<15} {'':<10}")

    print(f"\n{'Використання памяті':<35} {avg_mem_usage:.1f}{'':<15} {'МБ':<10}")
    print(f"{'Крок часу DT':<35} {DT:.3f}{'':<15} {'сек':<10}")
    print(f"{'Коефіцієнт CBF γ':<35} {GAMMA:.1f}{'':<15} {'':<10}")
    print("="*50)

if __name__ == "__main__":
    main_simulation()